# Проект: классификация

In [3524]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif

## Часть 1. Знакомство с данными, обработка пропусков и выбросов

### Задание 1

In [3525]:
df = pd.read_csv('bank_fin.csv', sep = ';')
df.head(10)

age         job   marital  education default       balance housing loan  \
0   59      admin.   married  secondary      no   2 343,00 $      yes   no   
1   56      admin.   married  secondary      no      45,00 $       no   no   
2   41  technician   married  secondary      no   1 270,00 $      yes   no   
3   55    services   married  secondary      no   2 476,00 $      yes   no   
4   54      admin.   married   tertiary      no     184,00 $       no   no   
5   42  management    single   tertiary      no         0  $      yes  yes   
6   56  management   married   tertiary      no     830,00 $      yes  yes   
7   60     retired  divorced  secondary      no     545,00 $      yes   no   
8   37  technician   married  secondary      no       1,00 $      yes   no   
9   28    services    single  secondary      no           NaN     yes   no   

   contact  day month  duration  campaign  pdays  previous poutcome deposit  
0  unknown    5   may      1042         1     -1         0  unknown     yes  
1  unknown    5   may      1467         1     -1         0  unknown     yes  
2  unknown    5   may      1389         1     -1         0  unknown     yes  
3  unknown    5   may       579         1     -1         0  unknown     yes  
4  unknown    5   may       673         2     -1         0  unknown     yes  
5  unknown    5   may       562         2     -1         0  unknown     yes  
6  unknown    6   may      1201         1     -1         0  unknown     yes  
7  unknown    6   may      1030         1     -1         0  unknown     yes  
8  unknown    6   may       608         1     -1         0  unknown     yes  
9  unknown    6   may      1297         3     -1         0  unknown     yes

In [3526]:
# исследуйте данные на предмет пропусков. Где есть пропущенные значения? Сколько их?
print(df.info(), '\n')
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11137 non-null  object
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(6), object(11)
memory usage: 1.4+ MB
None 

age           0
job           0
marital       0
education     0
default       0
balance     

### Задание 2

In [3527]:
# есть ли в признаке job пропущенные значения? Возможно, они обозначены каким-то специальным словом?
df['job'].value_counts()

management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
unknown            70
Name: job, dtype: int64

### Задание 3

In [3528]:
# преобразуйте признак balance таким образом, чтобы он корректно считывался, как вещественное число (float)
import re

def balance_extraction(some_str):
    if not some_str:
        return None
    else:
        res = re.sub(r'[$\s]','', str(some_str))
        res = re.sub(r'[,]','.', res)
        return float(res)
    
df['balance'] = df['balance'].apply(balance_extraction)
df['balance'].mean().round(3)


1529.129

### Задание 4

In [3529]:
# обработайте пропуски в признаки balance , заменив их на медианные значения по данному признаку
df= df.fillna( {'balance':df['balance'].median()})

df['balance'].mean().round(3)

1526.936

### Задание 5

In [3530]:
# обработайте пропуски в категориальных признаках: job и education, заменив их на модальные значения

df['job'] = df['job'].replace({'unknown' : df['job'].mode()[0]})
df['education'] = df['education'].replace({'unknown' : df['education'].mode()[0]})

df[(df['job'] == df['job'].value_counts().index[0]) & (df['education'] == df['education'].value_counts().index[0])]['balance'].mean().round(3)


1598.883

### Задание 6

In [3531]:
# удалите все выбросы для признака balance
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - left*iqr
    upper_bound = quartile_3 + right*iqr
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned, (lower_bound, upper_bound)

outliers, cleaned, bounds = outliers_iqr_mod(df, 'balance') #, left = 3, right = 3

print('Верхняя и нижняя границы поиска выбросов по методу Тьюки', bounds)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')



Верхняя и нижняя границы поиска выбросов по методу Тьюки (-2241.0, 4063.0)
Число выбросов по методу Тьюки: 1057


### Задание 7

In [3532]:
#Сколько объектов осталось после удаления всех выбросов?

to_drop = df[(df['balance'] < bounds[0]) | (df['balance'] > bounds[1])].index
df.drop(index=to_drop, inplace = True)
df.shape[0]

10105

## Часть 2:  Разведывательный анализ

### Задание 1

In [3533]:
# изучите соотношение классов в ваших данных на предмет несбалансированности, проиллюстрируйте результат
# ваш код

### Задания 2 и 3

In [3534]:
#рассчитайте описательные статистики для количественных переменных, проинтерпретируйте результат
#ваш код

### Задания 4 и 5

In [3535]:
#рассчитайте описательные статистики для категориальных переменных, проинтерпретируйте результат
#ваш код
#постройте визуализации, иллюстрирующие результаты

### Задание 6

In [3536]:
# Узнайте, для какого статуса предыдущей маркетинговой кампании успех в текущей превалирует над количеством неудач.
# ваш код

### Задание 7

In [3537]:
# узнайте, в каком месяце чаще всего отказывались от предложения открыть депозит
# ваш код

### Задание 8

In [3538]:
# создайте возрастные группы и определите, в каких группах более склонны открывать депозит, чем отказываться от предложения

### Задания 9 и 10

In [3539]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от семейного статуса

In [3540]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от образования

In [3541]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от вида профессиональной занятости

### Задание 11

In [3542]:
# постройте сводную таблицу, чтобы определить люди с каким образованием и семейным статусом наиболее многочисленны
#(если рассматривать тех, кто открыл депозит)

## Часть 3: преобразование данных

### Задание 1

In [3543]:
# преобразуйте уровни образования

### Задания 2 и 3

In [3544]:
# преобразуйте бинарные переменные в представление из нулей и единиц

### Задание 4

In [3545]:
# создайте дамми-переменные

### Задания 5 и 6

In [3546]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности

### Задания 7 и 8

In [3547]:
X = df.drop(['deposit'], axis=1)
y = df['deposit']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)

In [3548]:
# рассчитайте необходимые показатели

### Задание 9

In [3549]:
# с помощью SelectKBest отберите 15 наиболее подходящих признаков

### Задание 10

In [3550]:
# нормализуйте данные с помощью minmaxsxaler

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Задание 1

In [3551]:
# обучите логистическую регрессию и рассчитайте метрики качества

### Задания 2,3,4

In [3552]:
# обучите решающие деревья, настройте максимальную глубину

### Задание 5

In [3553]:
# подберите оптимальные параметры с помощью gridsearch

# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Задание 1

In [3554]:
# обучите на ваших данных случайный лес

### Задания 2 и 3

In [3555]:
# используйте для классификации градиентный бустинг и сравните качество со случайным лесом

### Задание 4

In [3556]:
# объедините уже известные вам алгоритмы с помощью стекинга 

### Задание 5

In [3557]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга

### Задания 6,7,8

In [3558]:
# реализуйте оптимизацию гиперпараметров с помощью Optuna